<a target="_blank" href="https://colab.research.google.com/github/JonathanLoscalzo/tabi-rag-2024/blob/main/TABI_Chatear_con_el_Libro_de_C%C3%A1tedra_(2024).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Instalar dependencias requeridas



### Ollama

In [1]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [4]:
!ls /etc/systemd/system/
!sudo systemctl daemon-reload
!sudo systemctl enable ollama

dbus-org.freedesktop.resolve1.service  multi-user.target.wants	sysinit.target.wants
default.target.wants		       ollama.service		timers.target.wants
getty.target.wants		       sudo.service
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [5]:
!ollama pull llama3.1:8b
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 1a4c3c319823... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [6]:
# !curl -X POST http://localhost:11434/api/generate -d '{ "model": "llama3.1:8b", "prompt":"Why is the sky blue?" }'

### Python libs


In [7]:
!pip install streamlit ollama langchain langchain_community langchain_experimental langchain-chroma bs4 chromadb pypdf rank_bm25 sentence_transformers tqdm

In [8]:
# checkeando ollama
import ollama
response = ollama.chat(model='llama3.1:8b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to us because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the atmosphere. Here's a simplified explanation:

1. **Sunlight enters Earth's atmosphere**: When the sun shines, it sends out all types of electromagnetic radiation, including visible light. This light travels through space and into our atmosphere.
2. **Light encounters gas molecules**: The light encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2), which make up about 99% of the Earth's atmosphere. These molecules are much smaller than the wavelength of visible light, so they can easily scatter the light in all directions.
3. **Scattering favors shorter wavelengths**: The scattering effect is more pronounced for shorter wavelengths (i.e., blue and violet light) because these wavelengths have a higher frequency and energy. As a result, the blue and violet parts of the spectrum are scattered more efficiently than the longer w

In [9]:
!mkdir assets

mkdir: cannot create directory ‘assets’: File exists


In [34]:
!wget 'https://drive.usercontent.google.com/u/0/uc?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download' -O assets/TABI.pdf

--2024-08-15 14:02:12--  https://drive.usercontent.google.com/u/0/uc?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.20.132, 2607:f8b0:400e:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.20.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.usercontent.google.com/uc?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download [following]
--2024-08-15 14:02:12--  https://drive.usercontent.google.com/uc?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download [following]
--2024-08-15 14:02:12--  https://drive.usercontent.google.com/download?id=1m2XnjrFPIcN7iidl00d8XjQ8QAh4WMTH&export=download
Reusing exist

## Code

In [35]:
from pprint import pprint
from typing import List

from langchain.retrievers import EnsembleRetriever
from langchain_chroma import Chroma

# from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os

template = """Utilice las siguientes piezas de contexto para responder la pregunta al final.
Si no sabe la respuesta, simplemente diga que no la sabe, no intente inventar una respuesta.
Utilice tres oraciones como máximo y mantenga la respuesta lo más concisa posible.
Di siempre "¡gracias por preguntar!" al final de la respuesta.

{context}

Pregunta: {question}

Respuesta:"""


class TabiPipeline:
    vector_store_persistent_path = "assets/vector_store"

    def __init__(
        self,
        book_path: str = "assets/TABI.pdf",
        ollama_embeddings: str = "nomic-embed-text",
        llm_model: str = "llama3.1:8b",
    ):
        self.embeddings = OllamaEmbeddings(model=ollama_embeddings)
        # Check if the file exists and print the absolute path for verification
        if not os.path.exists(book_path):
            raise FileNotFoundError(f"File not found at: {os.path.abspath(book_path)}")

        self.book_path = book_path
        self.vector_store: Chroma = Chroma(
            collection_name="tabi",
            embedding_function=self.embeddings,
            persist_directory=self.vector_store_persistent_path,
        )
        self.bm25_retriever: BM25Retriever = None
        self.llm = Ollama(
            model=llm_model,
            temperature=0.1,
            # repeat_last_n=-1,
            top_k=20,
            top_p=0.5,
        )

        # inicializar los índices
        self.create_book_bm25()
        self.create_book_vector_store()

    @property
    def loader(self):
        return PyPDFLoader(self.book_path)

    @property
    def ensemble_retriever(self):
        return EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.chroma_retriever],
            weights=[0.5, 0.5],
        )

    @property
    def chroma_retriever(self):
        return self.vector_store.as_retriever()

    def create_book_vector_store(
        self,
    ):
        if self.vector_store._client.get_collection("tabi").count() > 0:
            return

        pages = self.loader.load_and_split()
        self.vector_store.add_documents(pages)

    def create_book_bm25(self):
        pages = self.loader.load_and_split()
        self.bm25_retriever = BM25Retriever.from_documents(pages)

    def get_template_parsed(self, docs: List[Document]) -> str:
        template = """CONTEXTO {index}\n\n {content}"""
        return 10 * "_" + "\n\n".join(
            [
                template.format(content=doc.page_content, index=index)
                for index, doc in enumerate(docs)
            ]
        )

    def rag_with_chroma(self, question: str) -> dict:
        output = self.vector_store.similarity_search(question, k=3)

        return dict(
            question=question,
            response=self.llm.invoke(
                template.format(
                    context=self.get_template_parsed(output),
                    question=question,
                )
            ),
        )

    def rag_with_bm25(self, question: str) -> dict:
        output = self.bm25_retriever.invoke(
            question,
            config={"configurable": {"search_kwargs_bm25": {"k": 3}}},
        )

        return dict(
            question=question,
            response=self.llm.invoke(
                template.format(
                    context=self.get_template_parsed(output),
                    question=question,
                )
            ),
        )

    def rag_with_ensemble(self, question: str) -> dict:
        output = self.ensemble_retriever.invoke(question)

        return dict(
            question=question,
            response=self.llm.invoke(
                template.format(
                    context=self.get_template_parsed(output),
                    question=question,
                )
            ),
        )




In [36]:
!chmod 777 assets/*
!ls -lha assets


total 23M
drwxrwxrwx 4 root root 4.0K Aug 15 13:37 .
drwxr-xr-x 1 root root 4.0K Aug 15 14:01 ..
drwxr-xr-x 2 root root 4.0K Aug 15 13:29 .ipynb_checkpoints
-rwxrwxrwx 1 root root  23M Aug 15 13:55 TABI.pdf
drwxrwxrwx 3 root root 4.0K Aug 15 13:44 vector_store


In [ ]:
tp = TabiPipeline()

questions = [
    "¿Qué es machine learning?",
    "¿Qué es el Business Intelligence?",
    "¿Qué es una dimensión?",
    "¿Qué es un cubo?",
    "¿Quién es Ralph Kimball?",
]
answers = []
for question in tqdm(questions):
    pprint(dict(
            question=question,
            bm25_response=tp.rag_with_bm25(question)["response"],
            chroma_response=tp.rag_with_chroma(question)["response"],
        )
    )

<ipython-input-37-d224535036a1>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for question in tqdm(questions):


  0%|          | 0/5 [00:00<?, ?it/s]

{'bm25_response': 'No lo sé.\n\n¡Gracias por preguntar!',
 'chroma_response': 'El Machine Learning (ML) es un subconjunto del '
                    'aprendizaje automático que se enfoca en desarrollar '
                    'algoritmos capaces de aprender patrones y relaciones en '
                    'grandes conjuntos de datos, permitiendo a las máquinas '
                    'tomar decisiones o realizar predicciones sin ser '
                    'explícitamente programadas. ¡Gracias por preguntar!',
 'question': '¿Qué es machine learning?'}
{'bm25_response': 'Según el contexto, el Business Intelligence (BI) se refiere '
                  'a la capacidad de contar con información única, relevante, '
                  'centralizada y consolidada para tomar decisiones ágiles. '
                  'También involucra herramientas para el análisis en línea de '
                  'información y facilita al usuario el acceso a información '
                  'en forma dinámica e independiente